In [1]:
from fs_arm_4dof import Arm4DoF
import math
import logging
import time
import numpy as np
# logging.basicConfig(level=logging.INFO)
arm = Arm4DoF(device="COM12")

In [24]:
# 气泵打开
arm.pump_on()

In [3]:
# 气泵关闭
arm.pump_off()

In [2]:
# 设置舵机为阻尼模式
arm.set_damping(800)

In [3]:
# 获取舵机原始角度
arm.get_servo_angles()

[-2.1, -3.4, -52.1, 0.9]

In [7]:
# 获取关节弧度
thetas = arm.get_thetas()
# 转换为角度
theta1, theta2, theta3, theta4 = [math.degrees(theta) for theta in thetas]
print("机械臂关节弧度 theta1={:.1f} theta2={:.1f} theta3={:.1f} theta4={:.1f}(单位 °)".format(theta1, theta2, theta3, theta4))

机械臂关节弧度 theta1=-0.3 theta2=-87.9 theta3=91.4 theta4=1.3(单位 °)


In [8]:
# 获取末端的位姿
(xe,ye,ze), pitch = arm.get_tool_pose()
print("末端的位置: x={:.2f} y={:.2f} z={:.2f}(单位cm) ".format(xe, ye, ze))
print("俯仰角: {:.1f}".format(math.degrees(pitch)))

末端的位置: x=14.13 y=-0.08 z=1.98(单位cm) 
俯仰角: 4.9


 测试逆向运动学

In [13]:
# 获取关节角度
thetas = arm.get_thetas()
print("Thetas: {}".format(thetas))
# 正向运动学
tool_posi,pitch = arm.forward_kinematics(thetas)

print("Tool Posi = {}".format(tool_posi))
print("Pitch = {}".format(pitch))

ret,thetas = arm.inverse_kinematics(tool_posi, pitch)

print("逆向运动学的结果")
print(thetas)

Thetas: [-0.9451758896065908, -2.3880349562760252, 2.3369568354978383, 0.2901802948835705]
Tool Posi = [4.149963822811005, -5.7444649318675, -0.5250161732082272]
Pitch = 0.23910217410538354
逆向运动学的结果
[-0.9451758896065908, -2.3880349562760257, 2.3369568354978383, 0.29018029488357094]


In [56]:
tool_posi = [5.0, 12.0, 0.0]
pitch = 0 # math.pi/6
interval = 0.5
arm.move_p2p_minjerk(tool_posi, pitch, interval)

In [55]:
tool_posi = [12.0, 5.0, 5.0]
pitch = 0.0
interval = 0.5
arm.move_p2p_minjerk(tool_posi, pitch, interval)

In [54]:
tool_posi = [8, -8, -1.0]
pitch = 0 # math.pi/6
interval = 0.4
arm.move_p2p_minjerk(tool_posi, pitch, interval)

测试Task Space下的Minimum Jerk
> 效果其实不太好

In [26]:
def action0():
    tool_posi = [15.0, 0, 5.0]
    pitch = 0 # math.pi/6
    T = 0.45
    arm.move_p2p_minjerk_in_tasksapce(tool_posi, pitch, T)
    
def action1():
    tool_posi = [12.0, 9.0, -6.0]
    pitch = 0 # math.pi/6
    T = 0.80
    arm.move_p2p_minjerk_in_tasksapce(tool_posi, pitch, T)

def action2():
    tool_posi = [12.0, -2.0, -6.0]
    pitch = 0 # math.pi/6
    T = 0.45
    arm.move_p2p_minjerk_in_tasksapce(tool_posi, pitch, T)

In [34]:
for i in range(1):
    action0()
    time.sleep(0.5)
    action1()
    time.sleep(0.5)
    action2()
    time.sleep(0.5)

## 画笔模式

In [35]:
# 切换工具为画笔
arm.switch_tool('pen')

True

In [36]:
canvas_z = -4.0 # 画面的Z轴坐标  (4.5)
pen_pitch = math.radians(0.0)
pen_speed =  10 # 单位 cm/s

def pen_move(x, y, z):
    '''运动画笔'''
    global arm
    global pen_pitch
    global pen_speed
    # 估计周期
    tool_posi, pitch = arm.get_tool_pose()
    xs, ys, zs = tool_posi
    d = math.sqrt((x-xs)**2 + (y-ys)**2 + (z-zs)**2)
    arm.move_p2p_minjerk_in_tasksapce([x, y, z], pen_pitch, d / pen_speed)
    # arm.move_line([x, y, z], pen_pitch)
    
def pen_up():
    '''抬笔'''
    global arm
    global pen_pitch
    tool_posi, pitch = arm.get_tool_pose()
    tool_posi[2] = canvas_z + 3

def pen_down():
    '''落笔'''
    global arm
    global pen_pitch
    tool_posi, pitch = arm.get_tool_pose()
    tool_posi[2] = canvas_z
    arm.move_p2p_minjerk_in_tasksapce(tool_posi, pen_pitch, 0.2)

def draw_line(x0, y0, x1, y1):
    global arm
    global canvas_z
    global pen_pitch
    
    pen_move(x0, y0, canvas_z+3)
    time.sleep(0.1)
    pen_move(x0, y0, canvas_z)
    time.sleep(0.1)
    pen_move(x1, y1, canvas_z)
    time.sleep(0.1)
    pen_move(x1, y1, canvas_z+3)
    time.sleep(0.1)
    
def draw_cross(cx, cy, width):
    pass
    
def draw_rect(cx, cy, w, h):
    global arm
    global canvas_z
    global pen_pitch
    
    T = 0.15
    arm.move_p2p_minjerk_in_tasksapce([cx+w/2, cy-h/2, canvas_z+1], pen_pitch, T)
    pen_down()
    
    arm.move_p2p_minjerk_in_tasksapce([cx+w/2, cy-h/2, canvas_z], pen_pitch, T)
    arm.move_p2p_minjerk_in_tasksapce([cx+w/2, cy+h/2, canvas_z], pen_pitch, T)
    arm.move_p2p_minjerk_in_tasksapce([cx-w/2, cy+h/2, canvas_z], pen_pitch, T)
    arm.move_p2p_minjerk_in_tasksapce([cx-w/2, cy-h/2, canvas_z], pen_pitch, T)
    arm.move_p2p_minjerk_in_tasksapce([cx+w/2, cy-h/2, canvas_z], pen_pitch, T)
    pen_up()
    
    



测试绘制直线

In [43]:
draw_line(21, 3, 18, -3)
draw_line(21, -3, 18, 3)

测试绘制网格

In [41]:
def draw_grid():
    # 绘制格子棋盘
    for x in np.linspace(20, 16, 4):
        draw_line(x, -6, x, 6)
        
    for y in np.linspace(6, -6, 4):
        draw_line(20, y, 16, y)

draw_grid()

In [31]:
arm.joint2servo_k

[-1.0626835448399374,
 0.9743790233942269,
 -1.0440285566574914,
 -0.9367035128274274]

In [32]:
arm.joint2servo_b

[-1.1000000000000003, 93.81490601354493, 45.99655783383952, 10.849716670016631]

测试绘制矩形

In [9]:
# draw_rect(16, 0, 3, 3)
# draw_rect(17, 3, 3, 3)
draw_rect(19, -6, 3, 3)
draw_rect(19, -3, 3, 3)
draw_rect(19, 0, 3, 3)
draw_rect(19, 3, 3, 3)
draw_rect(19, 6, 3, 3)

In [20]:
draw_rect(18, -6, 3, 3)
draw_rect(18, -3, 3, 3)
draw_rect(18, 0, 3, 3)
draw_rect(18, 3, 3, 3)
draw_rect(18, 6, 3, 3)